In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)
z
def replacer(string, char):
    pattern = char + "{2,}"
    string = re.sub(pattern, char, string)
    return string

# validates the column data
def col_header_val(df, table_config):
    """
    replace whitespaces in the column and standardized column names
    """
    df.columns = df.columns.str.lower().str.strip().str.replace(r'\s+', '_')
    expected_col = list(map(str.lower, table_config['columns']))
    expected_col.sort()
    df.columns = df.columns.str.lower()
    df = df.reindex(sorted(df.columns), axis=1)

    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("Column name and column length validation passed")
        return 1
    else:
        print("Column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file:", mismatched_columns_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f"expected columns: {expected_col}")
        return 0


Overwriting testutility.py


In [72]:
code = '''
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + "{2,}"
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df, table_config):
    """
    replace whitespaces in the column and standardized column names
    """
    df.columns = df.columns.str.lower().str.strip().str.replace(r'\s+', '_')

    expected_col = list(map(str.lower, table_config['columns']))
    expected_col.sort()

    df = df.reindex(sorted(df.columns), axis=1)

    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("Column name and column length validation passed")
        return 1
    else:
        print("Column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file:", mismatched_columns_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f"expected columns: {expected_col}")
        return 0
'''

with open('testutility.py', 'w') as file:
    file.write(code)


SyntaxError: ignored

Write YAML File


In [110]:
%%writefile file.yaml
file_type: csv
dataset_name : testfile
file_name : Animation
table_name : edsurv
inbound_delimiter: ","
skip_leading_rows: 1
columns: ['animation_score', 'favorites', 'genre', 'rank', 'reviewers', 'studio', 'source', 'animeid', 'members', 'episodes', 'popularity', 'name', 'producers', 'airing']


Overwriting file.yaml


In [111]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [112]:
config_data["file_type"]

'csv'

In [113]:
config_data


{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'Animation',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'skip_leading_rows': 1,
 'columns': ['animation_score',
  'favorites',
  'genre',
  'rank',
  'reviewers',
  'studio',
  'source',
  'animeid',
  'members',
  'episodes',
  'popularity',
  'name',
  'producers',
  'airing']}

In [114]:
import pandas as np
df_sample = pd.read_csv("Animation.csv", delimiter=",")
df_sample.head()

,animeID,name,source,producers,genre,studio,episodes,airing,rank,popularity,members,favorites,reviewers,Animation_score
0,1,Cowboy Bebop,Original,Bandai Visual,Action,Sunrise,26.0,False,26,39,795733,43460,405664,8.81
1,5,Cowboy Bebop: Tengoku no Tobira,Original,Sunrise,Action,Bones,1.0,False,164,449,197791,776,120243,8.41
2,6,Trigun,Manga,Victor Entertainment,Action,Madhouse,26.0,False,255,146,408548,10432,212537,8.30
3,7,Witch Hunter Robin,Original,Bandai Visual,Action,Sunrise,26.0,False,2371,1171,79397,537,32837,7.33
4,8,Bouken Ou Beet,Manga,TV Tokyo,Adventure,Toei Animation,52.0,False,3544,3704,11708,14,4894,7.03


In [116]:
#read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
print(source_file)
#print("", source_file)
df = pd.read_csv(source_file, delimiter=config_data['inbound_delimiter'])



./Animation.csv


In [117]:
with open(source_file, 'r') as file:
    first_line = file.readline()
    delimiters = [',', ';', '\t']  # common delimiters to check

    for delimiter in delimiters:
        if delimiter in first_line:
            print(f"The delimiter in the file is: {delimiter}")
            break
    else:
        print("Delimiter not found in the first line. Please check the file structure.")


The delimiter in the file is: ,


In [118]:
util.col_header_val(df, config_data)

Column name and column length validation passed


/content/testutility.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  """


1

In [119]:
print("columns of files are", df.columns)
print("columns of YAML are:", config_data["columns"])


columns of files are Index(['animeid', 'name', 'source', 'producers', 'genre', 'studio', 'episodes',
       'airing', 'rank', 'popularity', 'members', 'favorites', 'reviewers',
       'animation_score'],
      dtype='object')
columns of YAML are: ['animation_score', 'favorites', 'genre', 'rank', 'reviewers', 'studio', 'source', 'animeid', 'members', 'episodes', 'popularity', 'name', 'producers', 'airing']


In [120]:
if util.col_header_val(df, config_data)==0:
  print("validation failed")
else:
  print("col validation passed")

Column name and column length validation passed
col validation passed


/content/testutility.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [124]:
df = pd.read_csv("Animation.csv")


In [127]:


df.to_csv('output_file.csv.gz', sep='|', index=False, compression='gzip')


file_size = os.path.getsize('output_file.csv.gz')

num_rows = df.shape[0]
num_cols = df.shape[1]

# Print the summary
print(f"Summary of the file '{'output_file.csv.gz'}':")
print(f"Total number of rows: {num_rows}")
print(f"Total number of columns: {num_cols}")
print(f"File size: {file_size} bytes")


Summary of the file 'output_file.csv.gz':
Total number of rows: 13631
Total number of columns: 14
File size: 515711 bytes
